### import
- 입금순서 적용 방침
    - 기본은 직접입력
    - (최초 or 신고당시)이자채권 : 비용-이자-원금
    - 완제 : 비용-원금-이자 (완제자는 신복입금을 유지하는 것이 의미가 없고, 그동안 입금원금에 포함된 비용을 차감할 수 없었으므로 비용, 원금, 이자순서를 적용하는 게 낫겠다.)
- 직접입력 세부사항
    - 신복 입금원금은 전산에서 원금-비용 순으로 차감
    - 신복 이자, 기타채무는 전산에서 미수-연체 순으로 차감

In [2]:
import os, re
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
warnings.simplefilter("ignore")
pd.options.display.float_format = '{:,.0f}'.format

def get_previous_weekday_yymmdd(date):
    """날짜를 받아 이전 마지막 평일 날짜 yymmdd return"""
    # 전일
    date = date - timedelta(days=1)
    # 전일부터 하루씩 차감하면서 평일이면 return
    while date.weekday() > 4:  # 월요일은 0이고, 금요일은 4
        date -= timedelta(days=1)
    return date.strftime('%y%m%d')

def 파일명(폴더:str, 키워드:str, 부정키워드="") :
    file_list = os.listdir(폴더)
    if 부정키워드 == "" : 
        fn = [file for file in file_list if re.search(키워드, file)]
    else : 
        fn = [file for file in file_list if (re.search(키워드, file)!=None) & (re.search(부정키워드,file)==None)]
    if len(fn) == 1 :
        return join(폴더,fn[0])
    else :
        print("조건을 만족하는 파일이 둘 이상입니다.")
        
def 입금항목현재항목(입금순서:str) :
    """입금순서를 받아 입금항목명리스트와 현재항목명리스트 (2차원)를 반환"""
    입금순서 = re.sub('\d',"",입금순서)
    입금순서 = re.sub('법비용', "입금법비용", 입금순서)
    입금순서 = re.sub('미수', "입금미수이자", 입금순서)
    입금순서 = re.sub('연체', "입금연체이자", 입금순서)
    입금순서 = re.sub('원금', "입금원금", 입금순서)
    입금항목들 = 입금순서.split(' ')
    현재항목들 = [re.sub('입금', '현재', item) for item in 입금항목들]
    return 입금항목들, 현재항목들
        
        
credit_deposit_dtype = {"No.":str, "채무자키" : str, "계좌키":str, "담당자키":str, "타입금키":str, "납입회차":str, "계좌번호":str}
account_dtype = {'채무자키':str, '계좌키':str, '계좌번호':str}
완제회차 = ["999", "777", "666", "630"]

### 기본변수

In [5]:
# 어차피 계좌정보 읽으려면 입금창에서 키매칭 작업 해야함.
# 입금창에서 최초원금, 현재금액들 불어와 준다면 더욱 더 키매칭 작업을 하는게 효율적
# 입금경로1 : 신용회복변제금상환 파일 양식에 금액 조정 해주고, 입금순서 컬럼추가 > 신용회복 입금
# 입금경로2 : 신용회복 to 항목별입금 > 유틸리티 항목별입금
# 중복건 입금 순서는 No.도 아니고 회차도 아니고 행순서대로 먼저 적용됨. 따라서 회차별로 정렬하면 됨

################################
company = "솔림" # 솔림 or 대성
오류일단무시 = True # 오류가 있는 경우 기본입금순서에 따라 분배
workday = "" # 특별히 원하는 날짜 없으면 빈문자열로 남겨둘것
입금순서기본값 = "1법비용 2미수 3연체 4원금"
입금순서완제 = "1법비용 2원금 3미수 4연체"
sub_dir = "" # 2024
################################

# 입금일(최근 평일 or 선택일)
if workday == "" : 
    # 오늘 이전 중 가장 가까운 평일 날짜 가져오기
    workday = get_previous_weekday_yymmdd(datetime.today())

# 경로 및 파일명 : 키매칭 파일, 계좌 
wd = join(r"D:\3.자산\신용회복\신용회복 to 일반입금", company, sub_dir) # ""는 여러번 조인해도 상관없음
fn_credit_deposit = workday+"_일괄입금구분_"+company+".xlsx"
fn_account = workday+"_계좌_"+company+".xlsx"
print(workday)

240603


### 재작업 시작점

##### 1.파일읽기

In [43]:
# 오류 아니라서 직접입력 아니라 입금순서 적용할 필요가 있는 건 : 최초금액은 있으나, 채권신고당시 현재원금이 0원 / 허위신고의심but최초금액수정안하기로한것
temp = pd.read_excel(r'D:\3.자산\신용회복\신용회복 to 일반입금\입금순서적용할건.xlsx',sheet_name=company, dtype={"채무자키":str, "계좌키":str})
입금순서적용계좌키 = temp.계좌키.to_list()
print("입금순서 적용할 계좌키 수 : ", len(입금순서적용계좌키))

# 일괄입금구분파일(키매칭 신복입금파일), 계좌파일
cre_dep_ori = pd.read_excel(join(wd, fn_credit_deposit), dtype = credit_deposit_dtype).fillna("")
account_ori = pd.read_excel(join(wd, fn_account), dtype = account_dtype).fillna("")

# 타입금키에 중복있는지 체크
중복여부 = cre_dep_ori.타입금키.duplicated().unique()
if len(중복여부)==1 and 중복여부[0]==False : 
    print("타입금키 중복체크 이상무")
else : 
    print(cre_dep_ori.타입금키.duplicated().value_counts())

# 입금법비용 칼럼 만들기, 입금비고에 회차적기
cre_dep_ori.insert(25, "입금기타", cre_dep_ori.pop("입금기타"))
cre_dep_ori.insert(23, "입금법비용", 0)
cre_dep_ori["입금비고"] = cre_dep_ori["납입회차"]

입금순서 적용할 계좌키 수 :  57
타입금키 중복체크 이상무


##### 2.재분배 및 오류체크

In [46]:
cre_dep = cre_dep_ori.copy()
account = account_ori.copy()[["계좌키", "최초원금", "현재원금", "현재법비용", "현재미수이자", "현재연체이자", "현재합계"]]
# 입금파일 계좌키,납입회차 순으로 정렬
cre_dep.sort_values(["계좌키", "납입회차"], inplace=True)

# '계좌키' 열에서 각 값의 누적 빈도를 계산하여 새로운 열 '계좌키_누적빈도'에 추가
cre_dep['계좌키빈도'] = cre_dep.groupby('계좌키').cumcount() + 1

# 최초금액 및 현재금액 병합 - 인덱스 새로 부여됨
cre_dep = cre_dep.merge(account, how='left')
# 변수에 담으면 인덱스가 재설정되긴 하던데, 혹시 모르니 확실하게. 아래 반복문에서 i-1을 하려면 필수!
cre_dep.reset_index(drop=True, inplace=True)

# 직접입력 대신 입금순서 작성하기 - 이자채권류 : 최초원금 10원미만, 신고당시 이자채권--------------------
# 그러나 결국은 분배를 내가 해줘야 항목별입금을 사용할 수 있다.
입금순서cond = [(cre_dep.계좌키.isin(입금순서적용계좌키) | (cre_dep.최초원금<10)), cre_dep.납입회차.isin(완제회차)]
입금순서valus = [입금순서기본값, 입금순서완제]

cre_dep["입금순서"] = np.select(입금순서cond, 입금순서valus, default = cre_dep.입금순서)
#--------------------------------------------------------------------------------------------------


# 오류열 만들고 오늘자 입금 전에 이미 오류난 거 표시
cond = (cre_dep.현재원금<0) | (cre_dep.현재법비용<0) | (cre_dep.현재미수이자<0) | (cre_dep.현재연체이자<0)
cre_dep["오류"] = np.where(cond, "오류_이미오류", "")

for i, v in tqdm(cre_dep.iterrows(), total = len(cre_dep)) : 
    # 단건 또는 다건 중 가장 앞선 회차
    if v.계좌키빈도 == 1 :
        # 입금순서 적용건(이자채권, 완제건) : 입금액이 현재합계보다 많을 때만 오류
        if v.입금순서 != "직접입력" :
            # 오류 체크
            if v.입금합계 > v.현재합계 : cre_dep.loc[i, "오류"] = "오류_합계초과"
            # 정상건 : 입금분배 - 잔여합계가 0이 될때까지
            else :
                # 나우리 직접입력 분배건 초기화
                cre_dep.loc[i,["입금원금", "입금법비용", "입금미수이자", "입금연체이자"]] = 0
                
                # 입금총액(합계)를 순서에 따라 재작업
                잔여합계 = v.입금합계
                
                입금항목들, 현재항목들 = 입금항목현재항목(v.입금순서)
                for 입금항목, 현재항목 in zip(입금항목들, 현재항목들) : 
                    항목입금액 = min(cre_dep.loc[i,현재항목], 잔여합계)
                    # 항목별 입금액 배정
                    cre_dep.loc[i, 입금항목] = 항목입금액
                    # 잔여합계금 재계산
                    잔여합계 -= 항목입금액
                    # 잔여금이 0이면 종료
                    if 잔여합계 == 0 : break
                
        # 직접입력 : 입금미수이자와 연체이자는 나우리가 자동으로 나눈다. 혹시 -값이 있는지만 확인
        # 입금원금을 원금-비용 순으로 바꿔준다.(순서는 상관없다. 실효되면 원상회복 할거니까. 다만 실효전 부채증명서 발급시 비용은 감면이 안 돼서 오히려 유리)
        else : 
            # 원금 오류 체크 - 이자와 별도로 해야함
            if (v.입금원금 > v.현재원금) :
                입금법비용 = v.입금원금 - v.현재원금
                if 입금법비용 > v.현재법비용 : 
                    cre_dep.loc[i, "오류"] = "오류_원금초과"
                else : 
                    cre_dep.loc[i, "입금원금"] = cre_dep.loc[i, "현재원금"]
                    cre_dep.loc[i, "입금법비용"] = 입금법비용

            # 이자 오류 체크 
            if (v.입금미수이자 > v.현재미수이자) or (v.입금연체이자 > v.현재연체이자) : 
                cre_dep.loc[i, "오류"] = "오류_이자초과"
            
            
            
    # 다건 중 나중 회차 
    # 현재금액 = 전회차현재금액 - 전회차 입금액으로 수정필요, 정렬 했으므로 이전행의 값을 불러오면 됨
    else : 
        
        # 현재금액 수정------------------------------------------------------
        cre_dep.loc[i, "현재합계"] = cre_dep.loc[i-1, "현재합계"] - cre_dep.loc[i-1, "입금합계"]
        
        입금항목들, 현재항목들 = 입금항목현재항목(입금순서기본값) # 단시 원금 ~ 연체이자를 반복하기 위한 것으로 입금순서와 무관
        for 입금항목, 현재항목 in zip(입금항목들, 현재항목들) : 
            cre_dep.loc[i, 현재항목] = cre_dep.loc[i-1, 현재항목] - cre_dep.loc[i-1, 입금항목]
        # ------------------------------------------------------------------
            
        # 입금순서 적용건(이자채권, 완제건) : 입금액이 현재합계보다 많을 때만 오류
        if v.입금순서 != "직접입력" : 
            # 오류 체크
            if v.입금합계 > cre_dep.loc[i, "현재합계"] : 
                cre_dep.loc[i, "오류"] = "오류_합계초과"
            # 정상건 입금분배
            else :
                # 나우리 직접입력 분배건 초기화
                cre_dep.loc[i,["입금원금", "입금법비용", "입금미수이자", "입금연체이자"]] = 0
                
                # 분배 재작업
                잔여합계 = v.입금합계
                
                입금항목들, 현재항목들 = 입금항목현재항목(v.입금순서)
                for 입금항목, 현재항목 in zip(입금항목들, 현재항목들) : 
                    항목입금액 = min(cre_dep.loc[i,현재항목], 잔여합계)
                    # 항목별 입금액 배정
                    cre_dep.loc[i, 입금항목] = 항목입금액
                    # 잔여합계금 재계산
                    잔여합계 -= 항목입금액
                    # 잔여금이 0이면 종료
                    if 잔여합계 == 0 : break
        
        # 직접입력의 경우(나우리분배) : 나우리분배는 미수이자 연체이자로만 분배되므로 이것들만 체크 및 재분배하면 됨
        else :
            
            # 미수와 연체 재분배(체크할 오류x)
            실현재미수이자 = cre_dep.loc[i,"현재미수이자"]
            if 실현재미수이자 - v.입금미수이자 < 0 : 
                cre_dep.loc[i, "입금미수이자"] = 실현재미수이자 # 실제 남은 현재미수이자만큼만 입금, 나머지는 연체이자로
                cre_dep.loc[i, "입금연체이자"] = v.입금연체이자 - (실현재미수이자 - v.입금미수이자) # 차액 추가 -(-) = +
            
            
            # 원금 비용 재분배 및 오류체크 - 이자와 별도로 해야함
            if (v.입금원금 > v.현재원금) :
                입금법비용 = v.입금원금 - v.현재원금
                if 입금법비용 > v.현재법비용 : 
                    cre_dep.loc[i, "오류"] = "오류_원금초과"
                else : 
                    cre_dep.loc[i, "입금원금"] = cre_dep.loc[i, "현재원금"]
                    cre_dep.loc[i, "입금법비용"] = 입금법비용

            # 이자 오류 체크 
            if (cre_dep.loc[i, "입금연체이자"] > cre_dep.loc[i, "현재연체이자"]) : 
                cre_dep.loc[i, "오류"] = "오류_이자초과"
                
                
    # 오류 일단 무시하고 입금순서기본값으로 분배
    # 다건입금의 경우 현재 금액들 위에서 수정되었으므로 그대로 사용하면 됨
    if (오류일단무시) and (cre_dep.loc[i, "오류"].str.contains("오류")): # 현재합계를 초과하면 입금기타로
        
        # 분배 재작업
        잔여합계 = v.입금합계
        
        입금항목들, 현재항목들 = 입금항목현재항목(입금순서기본값)
        for 입금항목, 현재항목 in zip(입금항목들, 현재항목들) : 
            항목입금액 = min(cre_dep.loc[i,현재항목], 잔여합계)
            # 항목별 입금액 배정
            cre_dep.loc[i, 입금항목] = 항목입금액
            # 잔여합계금 재계산
            잔여합계 -= 항목입금액
            # 잔여금이 0이면 종료
            if 잔여합계 == 0 : break
        
        if 잔여합계 > 0 :
            cre_dep.loc[i, "입금기타"] = 잔여합계
                     

# 입금합계 재작성
cre_dep["합계비교"] = cre_dep.입금원금 + cre_dep.입금법비용 + cre_dep.입금미수이자 + cre_dep.입금연체이자 + cre_dep.입금기타

# 원본과 차이점 기록 및 재배정 오류 확인
cre_dep = cre_dep.merge(cre_dep_ori[["타입금키", "입금원금","입금법비용", "입금미수이자", "입금연체이자","입금기타","입금합계"]], on="타입금키", suffixes=["", "_원본"])
cre_dep["원금비교"] = cre_dep.입금원금 - cre_dep.입금원금_원본
cre_dep["법비용비교"] = cre_dep.입금법비용 - cre_dep.입금법비용_원본
cre_dep["미수비교"] = cre_dep.입금미수이자 - cre_dep.입금미수이자_원본
cre_dep["연체비교"] = cre_dep.입금연체이자 - cre_dep.입금연체이자_원본
cre_dep["기타비교"] = cre_dep.입금기타 - cre_dep.입금기타_원본
cre_dep["합계비교"] = cre_dep.입금합계 - cre_dep.입금합계_원본

# 수정항목있음
수정여부_cond = (cre_dep.원금비교 != 0) |(cre_dep.법비용비교 != 0)|(cre_dep.미수비교 != 0)|(cre_dep.연체비교 != 0)|(cre_dep.기타비교 != 0)|(cre_dep.합계비교 != 0)
cre_dep["수정여부"] = np.where(수정여부_cond, "수정", "미수정")

# 합계 다른 경우 : 무언가 잘못됐다.
합계이상 = cre_dep.query('합계비교 != 0')
if len(합계이상)>0 : 
    print("원본과 입금합계금액 다름!!")
    display(cre_dep.query('합계비교 != 0'))
else : 
    print("원본과 입금합계 비교 이상없음")

100%|██████████| 435/435 [00:00<00:00, 8782.00it/s]

오류!!! 합계비교 0이 아닌 건


,No.,채무자키,계좌키,담당자키,입금구분,입금순서,타입금키,입금메모,리파이낸싱,확인결과,...,입금미수이자_원본,입금연체이자_원본,입금기타_원본,입금합계_원본,원금비교,법비용비교,미수비교,연체비교,기타비교,수정여부


##### 3. 파일저장

In [48]:
# # 원본열 삭제 - 비교하러 보는 거니 삭제하지 말자
# cre_dep.drop(columns=["입금원금_원본", "입금비용_원본", "입금미수_원본", "입금연체_원본","입금합계_원본"])

# 수정, 오류 log 출력(입금순서 통일하기 전에 출력해야)
print("log파일을 출력합니다.")
cre_dep.to_excel(join(wd, workday + "_일괄입금구분_수정log_"+company+".xlsx"), index=False)

#-----------------------------------------------------------
항목별입금 = True # True로 할 것!!!! 
# False : 신복양식, True : 일반입금(항목별입금) 양식
# 신용회복은 중복건 미수,연체이자 분배 재작업한 것인 무용지물이 되어버리니 쓸수없다. 전산 입금파일 경로도 바뀌어서 귀찮을테고..
#-----------------------------------------------------------
입금출력여부 = False # 여기서 변경하는 거 아님

# 입금 파일 선언
if 항목별입금 : 
    cre_dep.입금순서 = "직접입력"
    # 일반입금파일 작성
    cols = ["채무자키", "계좌키", "입금일", "입금합계",'입금원금','입금법비용','입금미수이자', '입금연체이자', '입금기타',
            "입금구분", "입금자", "입금자구분", "입금은행", "입금계좌번호", "입금순서", "입금메모", "입금비고"]
    입금파일 = cre_dep[cols]
    파일명옵션 = "_항목별입금_"
# 신복양식
else : 
    cols = ['No.', '납입회차', '일시납감면율', '입금일', '지점명', '입금자',
        '주민번호', '지원구분', '입금자구분', '주채무자주민번호', '주채무자', '계좌번호', 
        '입금원금', '입금미수이자', '입금연체이자', '입금합계', '확정채권금액',  '입금계좌번호', '지로코드', '입금순서']
    입금파일 = cre_dep[cols]
    파일명옵션 = "__신용회복입금_"


# 오류건 체크 : 오류건 전산 수정 후 > 계좌 다시 다운
오류건수 = len(cre_dep.query('오류 != "" '))
if 오류건수 > 0 : 
    if 오류일단무시 : 
        print(오류건수, "건의 오류건이 있습니다. 입금 후 반드시 해당건을 확인하시기 바랍니다.----------------------")
        입금출력여부 = True
    else : 
        print('오류건 전산 수정 후 계좌 데이터를 다시 다운로드 하고 처음부터 재작업하세요----------------------')
else : 
    입금출력여부 = True
    print('오류건이 없습니다.')

# 출력 
if 입금출력여부 : 
    if 항목별입금 : 
        print("항목별입금 파일형식으로 출력합니다.")
        입금파일.to_excel(join(wd, workday + "_항목별입금_"+company+".xlsx"), index=False)
        
    else : 
        # 신복 양식 출력가능여부
        if cre_dep.입금법비용.sum() > 0 or cre_dep.입금기타.sum() > 0 : 
            print('법비용 또는 기타항목에 입금이 분배되어, 신용회복 양식의 입금파일을 출력할 수 없습니다.')
        else :
            print("신용회복 파일형식으로 출력합니다.")
            with pd.ExcelWriter(join(wd, workday + 파일명옵션 +company+".xlsx")) as writer:
                입금파일.to_excel(writer, index=False, startrow=2)        

log파일을 출력합니다.
1 건의 오류건이 있습니다.
항목별입금 파일형식으로 출력합니다.
